# Funadamental 15. Deep Learning with Numpy

딥러닝 프레임워크를 이용하면 몇 줄 안되는 코드만으로 MNIST 데이터셋을 99% 이상의 정확도로 분류할 수 있는 이미지 분류기를 만들 수 있으며, 이를 활용해서 다양한 카테고리의 이미지 분류기로 확장할 수 있다.

`Tensorflow`를 사용해 아래와 같이 MNIST 이미지 분류 모델을 구현 할 수 있다.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# MNIST 데이터를 로드. 다운로드하지 않았다면 다운로드까지 자동으로 진행됩니다. 
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()   

# 모델에 맞게 데이터 가공
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0
x_train_reshaped = x_train_norm.reshape(-1, x_train_norm.shape[1]*x_train_norm.shape[2])
x_test_reshaped = x_test_norm.reshape(-1, x_test_norm.shape[1]*x_test_norm.shape[2])

# 딥러닝 모델 구성 - 2 Layer Perceptron
model=keras.models.Sequential()
model.add(keras.layers.Dense(50, activation='sigmoid', input_shape=(784,)))  # 입력층 d=784, 은닉층 레이어 H=50
model.add(keras.layers.Dense(10, activation='softmax'))   # 출력층 레이어 K=10
model.summary()

# 모델 구성과 학습
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x_train_reshaped, y_train, epochs=10)

# 모델 테스트 결과
test_loss, test_accuracy = model.evaluate(x_test_reshaped,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

11493376/11490434 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                39250     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.8393 - accuracy: 0.8036
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2481 - accuracy: 0.9310
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1894 - accuracy: 0.9463
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1519 - accuracy: 0.9564
Epoch 5/10
1875/1875 [===================

인공신경망의 실제 구현 원리를 보다 명확하게 이해하기 위해, 그동안 들춰보지 않았던 프레임워크 내부에서 일어나는 일을 Numpy를 활용해 직접 구현해보자.

## Parameters, Weights

In [2]:
# 입력층 데이터의 모양(shape)
print(x_train_reshaped.shape)

# 테스트를 위해 x_train_reshaped의 앞 5개의 데이터를 가져온다.
X = x_train_reshaped[:5]
print(X.shape)

(60000, 784)
(5, 784)


In [3]:
weight_init_std = 0.1
input_size = 784
hidden_size=50

# 인접 레이어간 관계를 나타내는 파라미터 W를 생성하고 random 초기화
W1 = weight_init_std * np.random.randn(input_size, hidden_size)  
# 바이어스 파라미터 b를 생성하고 Zero로 초기화
b1 = np.zeros(hidden_size)

a1 = np.dot(X, W1) + b1   # 은닉층 출력

print(W1.shape)
print(b1.shape)
print(a1.shape)

(784, 50)
(50,)
(5, 50)


In [4]:
# 단일 레이어 구현 함수
def affine_layer_forward(X, W, b):
    y = np.dot(X, W) + b
    cache = (X, W, b)
    return y, cache

## Activation Functions

In [5]:
# 위 수식의 sigmoid 함수를 구현해 봅니다.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  


z1 = sigmoid(a1)
print(z1[0])  # sigmoid의 출력은 모든 element가 0에서 1사이

[0.69556247 0.69925109 0.56174916 0.72780905 0.86429043 0.10213618
 0.38738905 0.5686497  0.51880479 0.75338377 0.23301892 0.36254198
 0.8154763  0.40886509 0.75944316 0.74761219 0.71577437 0.89691049
 0.52897877 0.40309465 0.47135745 0.51769792 0.87789925 0.55200285
 0.21731846 0.72370001 0.61102982 0.65228327 0.14938993 0.63207113
 0.57110423 0.39295202 0.66203592 0.53727104 0.59133225 0.36023375
 0.32737551 0.49086042 0.27009251 0.87221474 0.25451675 0.44943155
 0.40651628 0.46588025 0.62336924 0.43222496 0.50261781 0.62065791
 0.79551363 0.26051261]


sigmoid 다음에 다시 Dense 레이어가 출현한다. 출력 노드 개수만 다를 뿐 동일한 구조입니다. 그렇다면 MLP 레이어를 아래와 같이 함수로 구현할 수 있다.

In [6]:
# 단일 레이어 구현 함수
def affine_layer_forward(X, W, b):
    y = np.dot(X, W) + b
    cache = (X, W, b)
    return y, cache

In [7]:
input_size = 784
hidden_size = 50
output_size = 10

W1 = weight_init_std * np.random.randn(input_size, hidden_size)
b1 = np.zeros(hidden_size)
W2 = weight_init_std * np.random.randn(hidden_size, output_size)
b2 = np.zeros(output_size)

a1, cache1 = affine_layer_forward(X, W1, b1)
z1 = sigmoid(a1)
a2, cache2 = affine_layer_forward(z1, W2, b2)    # z1이 다시 두번째 레이어의 입력이 됩니다. 

print(a2[0])  # 최종 출력이 output_size만큼의 벡터가 되었습니다.

[ 0.23803018 -0.60876001 -0.09301643  0.00427339 -0.33886057 -0.27702036
 -0.20515165 -0.10377825 -0.26617169  0.35262371]


최종 출력인 `a2`에 softmax 함수를 적용하면 모델의 출력은 입력 X가 10가지 숫자 중 하나일 확률의 형태로 가공된다.

In [8]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

In [9]:
y_hat = softmax(a2)
y_hat[0]  

array([0.13936297, 0.05975732, 0.10008654, 0.11031336, 0.07827209,
       0.08326525, 0.0894697 , 0.0990152 , 0.08417349, 0.15628408])

In [10]:
# 정답 라벨을 One-hot 인코딩하는 함수
def _change_one_hot_label(X, num_category):
    T = np.zeros((X.size, num_category))
    for idx, row in enumerate(T):
        row[X[idx]] = 1
        
    return T

Y_digit = y_train[:5]
t = _change_one_hot_label(Y_digit, 10)
t     # 정답 라벨의 One-hot 인코딩

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [11]:
print(y_hat[0])
print(t[0])

[0.13936297 0.05975732 0.10008654 0.11031336 0.07827209 0.08326525
 0.0894697  0.0990152  0.08417349 0.15628408]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [12]:
batch_num = y_hat.shape[0]
dy = (y_hat - t) / batch_num
dy    # softmax값의 출력으로 Loss를 미분한 값

array([[ 0.02787259,  0.01195146,  0.02001731,  0.02206267,  0.01565442,
        -0.18334695,  0.01789394,  0.01980304,  0.0168347 ,  0.03125682],
       [-0.16860897,  0.0127809 ,  0.02554674,  0.01674058,  0.01574358,
         0.01688283,  0.01991354,  0.01983426,  0.01429333,  0.0268732 ],
       [ 0.0336053 ,  0.01313015,  0.02496869,  0.0198159 , -0.18488307,
         0.01177344,  0.02189483,  0.02090308,  0.01677832,  0.02201337],
       [ 0.03219655, -0.18743863,  0.02324371,  0.01745701,  0.0123658 ,
         0.0170216 ,  0.02011995,  0.02303345,  0.01526985,  0.02673071],
       [ 0.03296502,  0.01113746,  0.02311573,  0.02145808,  0.01393631,
         0.01220957,  0.02594043,  0.02212202,  0.01409614, -0.17698075]])